In [ ]:
# -- IMPORTS START --
import pandas as pd
import glob
import re
import os
import sys
import pickle
import datetime
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from sklearn import tree, metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from scipy.signal import butter, filtfilt, find_peaks
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.model_selection import train_test_split
# -- IMPORTS END --

# enable zooming into graphs
%matplotlib notebook
plt.rcParams['figure.figsize'] = [9, 6] # width, height in inches

Calculates magnitude of acceleration and rotational rate

In [ ]:
def calc_magnitude(data):

    # Calculate magnitude
    data['accel_mag'] = np.sqrt(data['accelerationX']**2 + data['accelerationY']**2 + data['accelerationZ']**2) # absolute accel magnitude
    data['accel_mag'] = data['accel_mag'] - data['accel_mag'].mean() # detrend: "remove gravity"

    return data

In [ ]:
def rotation_magnitude(data):
    
    #Calculate rotation magnitude
    data['rotation_mag'] = np.sqrt(data['rotationRateX']**2 + data['rotationRateY']**2 + data['rotationRateZ']**2)
    data['rotation_mag'] = data['rotation_mag'] - data['rotation_mag'].mean() # detrend: "remove gravity"
    
    return data

Remove Noise from both Acceration and Rotation

In [ ]:
def remove_noise(data,sampling_rate):
    from scipy.signal import butter, filtfilt, find_peaks

    # Low pass filter
    cutoff = 5 # Hz
    order = 2
    b, a = butter(order, cutoff/(sampling_rate/2), btype='lowpass')
    data['filtered_accel_mag'] = filtfilt(b, a, data['accel_mag'])

    return data

In [ ]:
def remove_noise_rotation(data,sampling_rate):
    from scipy.signal import butter, filtfilt, find_peaks

    # Low pass filter
    cutoff = 5 # Hz
    order = 2
    b, a = butter(order, cutoff/(sampling_rate/2), btype='lowpass')
    data['filtered_rotation_mag'] = filtfilt(b, a, data['rotation_mag'])

    return data

Adding features per window with both acceleration and rotation

In [ ]:
#Do not modify
def add_features(window):
    features = {}
    # A = acceleration
    features['Aavg'] = window['filtered_accel_mag'].mean()
    features['Amax'] = window['filtered_accel_mag'].quantile(1)
    features['Amed'] = window['filtered_accel_mag'].quantile(0.5)
    features['Amin'] = window['filtered_accel_mag'].quantile(0)
    features['Aq25'] = window['filtered_accel_mag'].quantile(0.25)
    features['Aq75'] = window['filtered_accel_mag'].quantile(0.75)
    features['Astd'] = window['filtered_accel_mag'].std()
    # R = rotation
    features['Ravg'] = window['filtered_rotation_mag'].mean()
    features['Rmax'] = window['filtered_rotation_mag'].quantile(1)
    features['Rmed'] = window['filtered_rotation_mag'].quantile(0.5)
    features['Rmin'] = window['filtered_rotation_mag'].quantile(0)
    features['Rq25'] = window['filtered_rotation_mag'].quantile(0.25)
    features['Rq75'] = window['filtered_rotation_mag'].quantile(0.75)
    features['Rstd'] = window['filtered_rotation_mag'].std()
    df = pd.DataFrame()
    df = df._append(features,ignore_index=True)
    return df

In [ ]:
def train_decision_tree(frames):
    # Extract feature columns
    X = frames[['Aavg', 'Amax', 'Amed', 'Amin', 'Aq25', 'Aq75', 'Astd', 'Ravg', 'Rmax', 'Rmed', 'Rmin', 'Rq25', 'Rq75', 'Rstd']]

    # Extract target column
    y = frames['activity']

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Create model
    dt_model = DecisionTreeClassifier(criterion='entropy',max_depth=5).fit(X_train, y_train)
    dt_pred = dt_model.predict(X_test)

    # Evaluate on test set
    acc = dt_model.score(X_test, y_test)
    dt_cm = confusion_matrix(y_test, dt_pred, labels=dt_model.classes_)
    print(classification_report(y_test, dt_pred))
    print("Accuracy on test set:", acc)

    return dt_model,dt_cm,acc

In [ ]:
# Function to extract windows and features
def extract_features(data, window_sec, sample_rate, activity):
    features_list = []
    window_size = f'{window_sec}S'
    data['time'] = pd.to_datetime(data['time'])
    resampled_data = data.resample(window_size, on='time')
    
    for t, window in resampled_data:
        features = add_features(window)
        features['activity'] = activity
        features_list.append(features)
    features_df = pd.concat(features_list, ignore_index=True)
    return features_df
